In [1]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Normalize the data.
from sklearn import preprocessing
from numpy.linalg import cholesky, det, lstsq
from scipy.optimize import minimize
from numpy.linalg import inv

def pass_arg(Xx, nsim, tr_size):
    
    # Compute the RMSE
    def root_mean_squared_error(y_true, y_pred):
        return np.sqrt(np.mean((y_pred-y_true)**2))

    def kernel(X1, X2, l=1.0, sigma_f=1.0):
        '''
        Isotropic squared exponential kernel. Computes 
        a covariance matrix from points in X1 and X2.

        Args:
            X1: Array of m points (m x d).
            X2: Array of n points (n x d).

        Returns:
            Covariance matrix (m x n).
        '''
        sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
        return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

    def posterior_predictive(X_s, X_train, Y_train, l=1.0, sigma_f=1.0, sigma_y=1e-8):
        '''  
        Computes the suffifient statistics of the GP posterior predictive distribution 
        from m training data X_train and Y_train and n new inputs X_s.

        Args:
            X_s: New input locations (n x d).
            X_train: Training locations (m x d).
            Y_train: Training targets (m x 1).
            l: Kernel length parameter.
            sigma_f: Kernel vertical variation parameter.
            sigma_y: Noise parameter.

        Returns:
            Posterior mean vector (n x d) and covariance matrix (n x n).
        '''
        K = kernel(X_train, X_train, l, sigma_f) + sigma_y**2 * np.eye(len(X_train))
        K_s = kernel(X_train, X_s, l, sigma_f)
        K_ss = kernel(X_s, X_s, l, sigma_f) + 1e-8 * np.eye(len(X_s))
        K_inv = inv(K)

        # Equation (4)
        mu_s = K_s.T.dot(K_inv).dot(Y_train)

        # Equation (5)
        cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)
        
        min_eig = np.min(np.real(np.linalg.eigvals(cov_s)))
        print("eig:",min_eig)
        if min_eig < 0:
            cov_s -= 10*min_eig * np.eye(*cov_s.shape)
            
        cov_s += 1e8*min_eig * np.eye(*cov_s.shape)
    
        return mu_s, cov_s


    def nll_fn(X_train, Y_train, noise=0, naive=False):
        '''
        Returns a function that computes the negative log marginal
        likelihood for training data X_train and Y_train and given 
        noise level.

        Args:
            X_train: training locations (m x d).
            Y_train: training targets (m x 1).
            noise: known noise level of Y_train.
            naive: if True use a naive implementation of Eq. (7), if 
                   False use a numerically more stable implementation. 

        Returns:
            Minimization objective.
        '''
        def nll_naive(theta):
            # Naive implementation of Eq. (7). Works well for the examples 
            # in this article but is numerically less stable compared to 
            # the implementation in nll_stable below.
            K = kernel(X_train, X_train, l=theta[0], sigma_f=theta[1]) + \
                noise**2 * np.eye(len(X_train))
            return 0.5 * np.log(det(K)) + \
                   0.5 * Y_train.T.dot(inv(K).dot(Y_train)) + \
                   0.5 * len(X_train) * np.log(2*np.pi)

        def nll_stable(theta):
            # Numerically more stable implementation of Eq. (7) as described
            # in http://www.gaussianprocess.org/gpml/chapters/RW2.pdf, Section
            # 2.2, Algorithm 2.1.
            K = kernel(X_train, X_train, l=theta[0], sigma_f=theta[1]) + \
                noise**2 * np.eye(len(X_train)) 
            K += 1e-6 * np.eye(*K.shape)
    #         print(np.linalg.eigvals(K))
            L = cholesky(K)

            return np.sum(np.log(np.diagonal(L))) + \
                   0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train)[0])[0]) + \
                   0.5 * len(X_train) * np.log(2*np.pi)

        if naive:
            return nll_naive
        else:
            return nll_stable



    # Load labeled data
    data = np.loadtxt('../data/labeled_data.dat')
    x_labeled = data[:, :2].astype(np.float64) # -2 because we do not need porosity predictions
    y_labeled = data[:, -2:-1].astype(np.float64) # dimensionless bond length and porosity measurements

    # normalize dataset with MinMaxScaler
    scaler = preprocessing.MinMaxScaler(feature_range=(0.0, 1.0))
    x_labeled = scaler.fit_transform(x_labeled)
    # y_labeled = scaler.fit_transform(y_labeled)

    tr_size = tr_size

    # train and test data
    trainX, trainY = x_labeled[:tr_size,:], y_labeled[:tr_size]
    # testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]
#     testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]


    # trainY = np.transpose(trainY)
    # testY = np.transpose(testY)

    # data_phyloss = np.loadtxt('../data/unlabeled_data_BK_constw_v2_1525.dat')
    # x_unlabeled = data_phyloss[:, :]

    # # initial porosity
    # initporo = x_unlabeled[:, -1]

    # x_unlabeled1 = x_unlabeled[:1303, :2]
    # x_unlabeled2 = x_unlabeled[-6:, :2]
    # x_unlabeled = np.vstack((x_unlabeled1,x_unlabeled2))

    # x_unlabeled = scaler.fit_transform(x_unlabeled)
    # init_poro1 = initporo[:1303]
    # init_poro2 = initporo[-6:]
    # init_poro = np.hstack((init_poro1,init_poro2))


    # Optimization
    res = minimize(nll_fn(trainX, trainY), [.35, .45], 
                   bounds=((1e-5, None), (1e-5, None)),
                   method='L-BFGS-B')

    mu_s, cov_s = posterior_predictive(Xx, trainX, trainY, *res.x)

    # print(f'After parameter optimization: l={res.x[0]:.2f} sigma_f={res.x[1]:.2f}')
    print(cov_s)
    samples = np.random.multivariate_normal(mu_s.ravel(), cov_s, nsim)
#     print("RMSE:", root_mean_squared_error(testY, samples))
    return samples

In [55]:
Xx = np.random.uniform(size=(3, 2))
ss = pass_arg(Xx, 1, 30)
# print(ss)

eig: 8.771637533611218e-06
[[ 8.77163817e+02 -6.15424039e-06 -9.19877974e-06]
 [-6.26810837e-06  8.77163807e+02 -4.77901277e-06]
 [-9.25711946e-06 -4.56368086e-06  8.77163764e+02]]


C:\Users\berkc\Miniconda3\envs\R\lib\site-packages\ipykernel_launcher.py:108: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
C:\Users\berkc\Miniconda3\envs\R\lib\site-packages\ipykernel_launcher.py:164: RuntimeWarning: covariance is not positive-semidefinite.


In [65]:
import scipy.spatial.distance as spdist

def covSEard(hyp=None, x=None, z=None, der=None):
    ''' Squared Exponential covariance function with Automatic Relevance Detemination
     (ARD) distance measure. The covariance function is parameterized as:
    
     k(x^p,x^q) = sf2 * exp(-(x^p - x^q)' * inv(P) * (x^p - x^q)/2)
    
     where the P matrix is diagonal with ARD parameters ell_1^2,...,ell_D^2, where
     D is the dimension of the input space and sf2 is the signal variance.
    
     The hyperparameters are:
    
     hyp = [ log(ell_1)
             log(ell_2)
             ...
             log(ell_D)
             log(sqrt(sf2)) ]
    '''
#     if hyp == None:                 # report number of parameters
#         return ['D + 1']            # USAGE: integer OR D_+_int (spaces are SIGNIFICANT)
    
    [n, D] = x.shape
    ell = 1/np.exp(hyp[0:D])        # characteristic length scale

    sf2 = np.exp(2.*hyp[D])         # signal variance
    tmp = np.dot(np.diag(ell),x.T).T
    A = spdist.cdist(tmp, tmp, 'sqeuclidean')
    A = sf2*np.exp(-0.5*A)  
        
#     if z == 'diag':
#         A = np.zeros((n,1))
#     elif z == None:
#         tmp = np.dot(np.diag(ell),x.T).T
#         A = spdist.cdist(tmp, tmp, 'sqeuclidean')
#     else:                           # compute covariance between data sets x and z
#         A = spdist.cdist(np.dot(np.diag(ell),x.T).T, np.dot(np.diag(ell),z.T).T, 'sqeuclidean') # cross covariances
 
#     A = sf2*np.exp(-0.5*A)
#     if not der == None:
#         if der < D:                 # compute derivative matrix wrt length scale parameters
#             if z == 'diag':
#                 A = A*0.
#             elif z == None:
#                 tmp = np.atleast_2d(x[:,der]).T/ell[der]
#                 A = A * spdist.cdist(tmp, tmp, 'sqeuclidean')
#             else:
#                 A = A * spdist.cdist(np.atleast_2d(x[:,der]).T/ell[der], np.atleast_2d(z[:,der]).T/ell[der], 'sqeuclidean')
#         elif der==D:                # compute derivative matrix wrt magnitude parameter
#             A = 2.*A
#         else:
#             raise Exception("Wrong derivative index in covSEard")
                
    return A

covSEard(hyp=[.5,.5,.5], x=trainX, z=trainX, der=None)

[[0.         0.41935484]
 [1.         0.64516129]
 [0.9        1.        ]
 [0.48       0.61290323]
 [0.26       0.4516129 ]]


array([[2.71828183, 2.24046058, 2.20118042, 2.58759721, 2.68417729],
       [2.24046058, 2.71828183, 2.65116896, 2.58589401, 2.44094204],
       [2.20118042, 2.65116896, 2.71828183, 2.55995783, 2.38534172],
       [2.58759721, 2.58589401, 2.55995783, 2.71828183, 2.68132803],
       [2.68417729, 2.44094204, 2.38534172, 2.68132803, 2.71828183]])

In [66]:
# # Load labeled data
# data = np.loadtxt('../data/labeled_data.dat')
# x_labeled = data[:, :2].astype(np.float64) # -2 because we do not need porosity predictions
# y_labeled = data[:, -2:-1].astype(np.float64) # dimensionless bond length and porosity measurements

# # normalize dataset with MinMaxScaler
# scaler = preprocessing.MinMaxScaler(feature_range=(0.0, 1.0))
# x_labeled = scaler.fit_transform(x_labeled)
# # y_labeled = scaler.fit_transform(y_labeled)

# tr_size = 5

# # train and test data
# trainX, trainY = x_labeled[:tr_size,:], y_labeled[:tr_size]


# def exponential_cov(x, y, params):
#     return params[0]**2 * np.exp( -0.5 * params[1] * np.subtract.outer(x, y)**2)

# def kernel(X1, X2, l=1.0, sigma_f=1.0):
#     sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
#     return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

# b1=kernel(trainX, trainX, 1.0,1.0)
# b2=exponential_cov(trainX, trainX, [1.0,1.0])
# print(b1)
# print(b2)


# # print(trainX**2)
# # a1 = np.sum(trainX**2, 1).reshape(-1, 1)
# # a2 = np.sum(trainX**2, 1) 
# # a3 = -2 * np.dot(trainX, trainX.T)
# # print(a1, a2, a3)

In [ ]:
import numpy as np

# Normalize the data.
from sklearn import preprocessing
from numpy.linalg import cholesky, det, lstsq
from scipy.optimize import minimize
import scipy.spatial.distance as spdist

def pass_arg(Xx, nsim, tr_size):
    
    log = np.log
    
    # Compute the RMSE
    def root_mean_squared_error(y_true, y_pred):
        return np.sqrt(np.mean((y_pred-y_true)**2))

    # Making sure final porosity is less than initial
    def poros(poroi, porof):
        porofn = -porof*(porof<0)
        porofp = porof*(porof>=poroi) - poroi*(porof>=poroi)
        return porofp+porofn


    # Load labeled data
    data = np.loadtxt('../data/labeled_data.dat')
    x_labeled = data[:, :2].astype(np.float64) # -2 because we do not need porosity predictions
    y_labeled = data[:, -2:-1].astype(np.float64) # dimensionless bond length and porosity measurements

    # normalize dataset with MinMaxScaler
    scaler = preprocessing.MinMaxScaler(feature_range=(0.0, 1.0))
    x_labeled = scaler.fit_transform(x_labeled)
    # y_labeled = scaler.fit_transform(y_labeled)

    tr_size = tr_size

    # train and test data
    trainX, trainY = x_labeled[:tr_size,:], y_labeled[:tr_size]
    testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]


    # trainY = np.transpose(trainY)
    # testY = np.transpose(testY)

    data_phyloss = np.loadtxt('../data/unlabeled_data_BK_constw_v2_1525.dat')
    x_unlabeled = data_phyloss[:, :]

    # initial porosity
    initporo = x_unlabeled[:, -1]

    x_unlabeled1 = x_unlabeled[:1303, :2]
    x_unlabeled2 = x_unlabeled[-6:, :2]
    x_unlabeled = np.vstack((x_unlabeled1,x_unlabeled2))

    x_unlabeled = scaler.fit_transform(x_unlabeled)
    init_poro1 = initporo[:1303]
    init_poro2 = initporo[-6:]
    init_poro = np.hstack((init_poro1,init_poro2))
    
    
    def covSEard(hyp=None, x=None, z=None):
        ''' Squared Exponential covariance function with Automatic Relevance Detemination
         (ARD) distance measure. The covariance function is parameterized as:

         k(x^p,x^q) = sf2 * exp(-(x^p - x^q)' * inv(P) * (x^p - x^q)/2)

         where the P matrix is diagonal with ARD parameters ell_1^2,...,ell_D^2, where
         D is the dimension of the input space and sf2 is the signal variance.

         The hyperparameters are:

         hyp = [ log(ell_1)
                 log(ell_2)
                 ...
                 log(ell_D)
                 log(sqrt(sf2)) ]
        '''

        [n, D] = x.shape
        ell = 1/np.array(hyp[0:D])        # characteristic length scale
        
        
        sf2 = np.array(hyp[D])**2         # signal variance
        tmp = np.dot(np.diag(ell),x.T).T
        
        if z == 'itself':
            tmp = np.dot(np.diag(ell),x.T).T
            A = spdist.cdist(tmp, tmp, 'sqeuclidean')
        else:                           # compute covariance between data sets x and z
            A = spdist.cdist(np.dot(np.diag(ell),x.T).T, np.dot(np.diag(ell),z.T).T, 'sqeuclidean') # cross covariances

        A = sf2*np.exp(-0.5*A)  

        return A


    def posterior_predictive(X_s, X_train, Y_train, l1=log(.1), l2=log(.1), sigma_f=log(.1), sigma_y=0):
        '''  
        Computes the suffifient statistics of the GP posterior predictive distribution 
        from m training data X_train and Y_train and n new inputs X_s.

        Args:
            X_s: New input locations (n x d).
            X_train: Training locations (m x d).
            Y_train: Training targets (m x 1).
            l: Kernel length parameter.
            sigma_f: Kernel vertical variation parameter.
            sigma_y: Noise parameter.

        Returns:
            Posterior mean vector (n x d) and covariance matrix (n x n).
        '''


        K = covSEard(hyp=[l1,l2,sigma_f], x=X_train, z='itself') + sigma_y**2 * np.eye(len(X_train))
        K_s = covSEard(hyp=[l1,l2,sigma_f], x=X_train, z=X_s)
        K_ss = covSEard(hyp=[l1,l2,sigma_f], x=X_s, z='itself')  + 1e-8 * np.eye(len(X_s))
#         K_inv = inv(K)
        K_inv = np.linalg.pinv(K)
    
        # Equation (4)
        mu_s = K_s.T.dot(K_inv).dot(Y_train)

        # Equation (5)
        cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)
        
        return mu_s, cov_s


    def nll_fn(X_train, Y_train, x_unlabeled, init_poro, noise=0, naive=False):
        '''
        Returns a function that computes the negative log marginal
        likelihood for training data X_train and Y_train and given 
        noise level.

        Args:
            X_train: training locations (m x d).
            Y_train: training targets (m x 1).
            noise: known noise level of Y_train.
            naive: if True use a naive implementation of Eq. (7), if 
                   False use a numerically more stable implementation. 

        Returns:
            Minimization objective.
        '''

        def nll_stable(theta):
            # Numerically more stable implementation of Eq. (7) as described
            # in http://www.gaussianprocess.org/gpml/chapters/RW2.pdf, Section
            # 2.2, Algorithm 2.1.
            K = covSEard(hyp=[theta[0],theta[1],theta[2]], x=X_train, z=X_train) + \
                noise**2 * np.eye(len(X_train))
            
            
            K += 1e-8 * np.eye(*K.shape)
            L = cholesky(K)
        

            mu_un, _ = posterior_predictive(x_unlabeled, X_train, Y_train, l1=theta[0], l2=theta[1], sigma_f=theta[2])
            phyloss_poro = np.mean(poros(init_poro, mu_un))

            log_loss = np.sum(np.log(np.diagonal(L))) + \
                   0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train)[0])[0]) + \
                   0.5 * len(X_train) * np.log(2*np.pi)
        
            print(500000*phyloss_poro,log_loss, theta)
            return 500000*phyloss_poro + log_loss

        if naive:
            return nll_naive
        else:
            return nll_stable

    
    # Optimization
    res = minimize(nll_fn(trainX, trainY, x_unlabeled, init_poro), x0 = [.1, .1, .1], 
                   bounds=((1e-5, None), (1e-5, None), (1e-5, None)),
                    method='L-BFGS-B')
    
#     print(f'After parameter optimization: l1={res.x[0]:.5f} l2={res.x[1]:.5f} sigma_f={res.x[2]:.5f}')
#     print(np.exp(res.x[0]),np.exp(res.x[1]), np.exp(res.x[2]))
    mu_s, cov_s = posterior_predictive(Xx, trainX, trainY, *res.x)
    
    samples = np.random.multivariate_normal(mu_s.ravel(), cov_s, nsim)
#     print("RMSE:", root_mean_squared_error(testY, samples))

    return samples

In [283]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Normalize the data.
from sklearn import preprocessing
from numpy.linalg import cholesky, det, lstsq
from scipy.optimize import minimize
from numpy.linalg import inv
import scipy.spatial.distance as spdist

def pass_arg(Xx, nsim, tr_size):
    
    log = np.log
    
    # Compute the RMSE
    def root_mean_squared_error(y_true, y_pred):
        return np.sqrt(np.mean((y_pred-y_true)**2))

    # Making sure final porosity is less than initial
    def poros(poroi, porof):
        porofn = -porof*(porof<0)
        porofp = porof*(porof>=poroi) - poroi*(porof>=poroi)
        return porofp+porofn


    # Load labeled data
    data = np.loadtxt('../data/labeled_data.dat')
    x_labeled = data[:, :2].astype(np.float64) # -2 because we do not need porosity predictions
    y_labeled = data[:, -2:-1].astype(np.float64) # dimensionless bond length and porosity measurements

    # normalize dataset with MinMaxScaler
    scaler = preprocessing.MinMaxScaler(feature_range=(0.0, 1.0))
    x_labeled = scaler.fit_transform(x_labeled)
    # y_labeled = scaler.fit_transform(y_labeled)

    tr_size = tr_size

    # train and test data
    trainX, trainY = x_labeled[:tr_size,:], y_labeled[:tr_size]
    testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]


    # trainY = np.transpose(trainY)
    # testY = np.transpose(testY)

    data_phyloss = np.loadtxt('../data/unlabeled_data_BK_constw_v2_1525.dat')
    x_unlabeled = data_phyloss[:, :]

    # initial porosity
    initporo = x_unlabeled[:, -1]

    x_unlabeled1 = x_unlabeled[:1303, :2]
    x_unlabeled2 = x_unlabeled[-6:, :2]
    x_unlabeled = np.vstack((x_unlabeled1,x_unlabeled2))

    x_unlabeled = scaler.fit_transform(x_unlabeled)
    init_poro1 = initporo[:1303]
    init_poro2 = initporo[-6:]
    init_poro = np.hstack((init_poro1,init_poro2))

    
    
    def covSEard(hyp=None, x=None, z=None):
        ''' Squared Exponential covariance function with Automatic Relevance Detemination
         (ARD) distance measure. The covariance function is parameterized as:

         k(x^p,x^q) = sf2 * exp(-(x^p - x^q)' * inv(P) * (x^p - x^q)/2)

         where the P matrix is diagonal with ARD parameters ell_1^2,...,ell_D^2, where
         D is the dimension of the input space and sf2 is the signal variance.

         The hyperparameters are:

         hyp = [ log(ell_1)
                 log(ell_2)
                 ...
                 log(ell_D)
                 log(sqrt(sf2)) ]
        '''
        #     if hyp == None:                 # report number of parameters
        #         return ['D + 1']            # USAGE: integer OR D_+_int (spaces are SIGNIFICANT)

        [n, D] = x.shape
        ell = 1/np.array(hyp[0:D])        # characteristic length scale
        
        
        sf2 = np.array(hyp[D])**2         # signal variance
        tmp = np.dot(np.diag(ell),x.T).T
#         A = spdist.cdist(tmp, tmp, 'sqeuclidean')
        
        if z == 'itself':
            tmp = np.dot(np.diag(ell),x.T).T
            A = spdist.cdist(tmp, tmp, 'sqeuclidean')
        else:                           # compute covariance between data sets x and z
            A = spdist.cdist(np.dot(np.diag(ell),x.T).T, np.dot(np.diag(ell),z.T).T, 'sqeuclidean') # cross covariances

        A = sf2*np.exp(-0.5*A)  

        return A

    
#     def kernel(X1, X2, l=1.0, sigma_f=1.0):
#         '''
#         Isotropic squared exponential kernel. Computes 
#         a covariance matrix from points in X1 and X2.

#         Args:
#             X1: Array of m points (m x d).
#             X2: Array of n points (n x d).

#         Returns:
#             Covariance matrix (m x n).
#         '''
#         sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
#         return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

    def posterior_predictive(X_s, X_train, Y_train, l1=log(.1), l2=log(.1), sigma_f=log(.1), sigma_y=0):
        '''  
        Computes the suffifient statistics of the GP posterior predictive distribution 
        from m training data X_train and Y_train and n new inputs X_s.

        Args:
            X_s: New input locations (n x d).
            X_train: Training locations (m x d).
            Y_train: Training targets (m x 1).
            l: Kernel length parameter.
            sigma_f: Kernel vertical variation parameter.
            sigma_y: Noise parameter.

        Returns:
            Posterior mean vector (n x d) and covariance matrix (n x n).
        '''
        
#         K = kernel(X_train, X_train, l, sigma_f) + sigma_y**2 * np.eye(len(X_train))
#         K_s = kernel(X_train, X_s, l, sigma_f)
#         K_ss = kernel(X_s, X_s, l, sigma_f) + 1e-8 * np.eye(len(X_s))
#         K_inv = inv(K)


        K = covSEard(hyp=[l1,l2,sigma_f], x=X_train, z='itself') + sigma_y**2 * np.eye(len(X_train))
        K_s = covSEard(hyp=[l1,l2,sigma_f], x=X_train, z=X_s)
        K_ss = covSEard(hyp=[l1,l2,sigma_f], x=X_s, z='itself')  + 1e-8 * np.eye(len(X_s))
#         K_inv = inv(K)
        K_inv = np.linalg.pinv(K)
    
        # Equation (4)
        mu_s = K_s.T.dot(K_inv).dot(Y_train)

        # Equation (5)
#         print(K_s.T.dot(K_inv).shape, K_s.T.dot(K_inv).dot(K_s).shape, K_ss.shape)
        cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)
        
#         min_eig = np.min(np.real(np.linalg.eigvals(cov_s)))
# #         print("eig:",min_eig)
#         if min_eig < 0:
#             cov_s -= 10*min_eig * np.eye(*cov_s.shape)
            
#         cov_s += 1e8*min_eig * np.eye(*cov_s.shape)
    
        return mu_s, cov_s


    def nll_fn(X_train, Y_train, x_unlabeled, init_poro, noise=0, naive=False):
        '''
        Returns a function that computes the negative log marginal
        likelihood for training data X_train and Y_train and given 
        noise level.

        Args:
            X_train: training locations (m x d).
            Y_train: training targets (m x 1).
            noise: known noise level of Y_train.
            naive: if True use a naive implementation of Eq. (7), if 
                   False use a numerically more stable implementation. 

        Returns:
            Minimization objective.
        '''
#         def nll_naive(theta):
#             # Naive implementation of Eq. (7). Works well for the examples 
#             # in this article but is numerically less stable compared to 
#             # the implementation in nll_stable below.
#             K = kernel(X_train, X_train, l=theta[0], sigma_f=theta[1]) + \
#                 noise**2 * np.eye(len(X_train))
#             return 0.5 * np.log(det(K)) + \
#                    0.5 * Y_train.T.dot(inv(K).dot(Y_train)) + \
#                    0.5 * len(X_train) * np.log(2*np.pi)

        def nll_stable(theta):
            # Numerically more stable implementation of Eq. (7) as described
            # in http://www.gaussianprocess.org/gpml/chapters/RW2.pdf, Section
            # 2.2, Algorithm 2.1.
#             K = kernel(X_train, X_train, l=theta[0], sigma_f=theta[1]) + \
#                 noise**2 * np.eye(len(X_train)) 
            K = covSEard(hyp=[theta[0],theta[1],theta[2]], x=X_train, z=X_train) + \
                noise**2 * np.eye(len(X_train))
            
            
            K += 1e-8 * np.eye(*K.shape)
    #         print(np.linalg.eigvals(K))
            L = cholesky(K)
        

            mu_un, _ = posterior_predictive(x_unlabeled, X_train, Y_train, l1=theta[0], l2=theta[1], sigma_f=theta[2])
            phyloss_poro = np.mean(poros(init_poro, mu_un))
            
#             print("normal loss:", np.sum(np.log(np.diagonal(L))) + \
#                    0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train)[0])[0]) + \
#                    0.5 * len(X_train) * np.log(2*np.pi))
#             
#             return np.sum(np.log(np.diagonal(L))) + \
#                    0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train)[0])[0]) + \
#                    0.5 * len(X_train) * np.log(2*np.pi)

            log_loss = np.sum(np.log(np.diagonal(L))) + \
                   0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train)[0])[0]) + \
                   0.5 * len(X_train) * np.log(2*np.pi)
            

#             print(log_loss, theta)
#             return log_loss
        
            print(500000*phyloss_poro,log_loss, theta)
            return 500000*phyloss_poro + log_loss

        if naive:
            return nll_naive
        else:
            return nll_stable

    
    # Optimization
    res = minimize(nll_fn(trainX, trainY, x_unlabeled, init_poro), x0 = [.1, .1, .1], 
                   bounds=((1e-5, None), (1e-5, None), (1e-5, None)),
                    method='L-BFGS-B')
    
    print(f'After parameter optimization: l1={res.x[0]:.5f} l2={res.x[1]:.5f} sigma_f={res.x[2]:.5f}')
    print(np.exp(res.x[0]),np.exp(res.x[1]), np.exp(res.x[2]))
    mu_s, cov_s = posterior_predictive(Xx, trainX, trainY, *res.x)
    
    samples = np.random.multivariate_normal(mu_s.ravel(), cov_s, nsim)
#     print("RMSE:", root_mean_squared_error(testY, samples))

    return samples

In [284]:
Xx = np.random.uniform(size=(100, 2))
ss = pass_arg(Xx, 1, 30)
# print(ss)
# print(mu)

C:\Users\berkc\Miniconda3\envs\R\lib\site-packages\ipykernel_launcher.py:94: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
C:\Users\berkc\Miniconda3\envs\R\lib\site-packages\ipykernel_launcher.py:219: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


0.0 [[26902.83334319]] [0.1 0.1 0.1]
0.0 [[26902.83334299]] [0.10000001 0.1        0.1       ]
0.0 [[26902.83334263]] [0.1        0.10000001 0.1       ]
0.0 [[26902.83334491]] [0.1        0.1        0.10000001]
11545.065826339196 [[27871.65239384]] [0.43935192 1.04065801 0.09833507]
11545.060611781093 [[27871.65241651]] [0.43935193 1.04065801 0.09833507]
11545.069846810109 [[27871.65241024]] [0.43935192 1.04065802 0.09833507]
11545.06542184991 [[27871.65233146]] [0.43935192 1.04065801 0.09833508]
2276.3439604011874 [[26902.68495692]] [0.20558832 0.39268287 0.09948196]
2276.343883240979 [[26902.68495743]] [0.20558833 0.39268287 0.09948196]
2276.3441482694934 [[26902.68496024]] [0.20558832 0.39268288 0.09948196]
2276.3439604174528 [[26902.684955]] [0.20558832 0.39268287 0.09948197]
0.0 [[26902.76473677]] [0.1004541  0.10125873 0.09999777]
0.0 [[26902.764736]] [0.10045411 0.10125873 0.09999777]
0.0 [[26902.76473531]] [0.1004541  0.10125874 0.09999777]
0.0 [[26902.7647379]] [0.1004541  0.1

0.0 [[26881.26192636]] [0.09863436 0.22032601 0.02638122]
0.0 [[26881.26192654]] [0.09863435 0.22032602 0.02638122]
0.0 [[26881.26192654]] [0.09863435 0.22032601 0.02638123]
0.1965968556512684 [[26881.23040025]] [0.10054487 0.22044773 0.02629536]
0.19660041339532455 [[26881.2304001]] [0.10054488 0.22044773 0.02629536]
0.19659737205310085 [[26881.23040028]] [0.10054487 0.22044774 0.02629536]
0.196596855652037 [[26881.23040016]] [0.10054487 0.22044773 0.02629537]
0.0 [[26881.2551953]] [0.09902602 0.22035096 0.02636361]
0.0 [[26881.25519508]] [0.09902603 0.22035096 0.02636361]
0.0 [[26881.2551953]] [0.09902602 0.22035097 0.02636361]
0.0 [[26881.25519522]] [0.09902602 0.22035096 0.02636362]
0.0 [[26881.25081498]] [0.09928536 0.22036749 0.02635196]
0.0 [[26881.25081483]] [0.09928537 0.22036749 0.02635196]
0.0 [[26881.25081504]] [0.09928536 0.2203675  0.02635196]
0.0 [[26881.25081498]] [0.09928536 0.22036749 0.02635197]
35.17696992734038 [[26881.00796764]] [0.11649679 0.2165704  0.02568515]


0.0 [[26881.23125954]] [0.10038248 0.21571441 0.0260784 ]
0.0 [[26881.23125626]] [0.10037962 0.21573118 0.02608121]
0.0 [[26881.23125609]] [0.10037963 0.21573118 0.02608121]
0.0 [[26881.23125624]] [0.10037962 0.21573119 0.02608121]
0.0 [[26881.2312562]] [0.10037962 0.21573118 0.02608122]
0.0 [[26881.2312561]] [0.10037944 0.21573223 0.02608139]
0.0 [[26881.23125591]] [0.10037945 0.21573223 0.02608139]
0.0 [[26881.23125608]] [0.10037944 0.21573224 0.02608139]
0.0 [[26881.23125608]] [0.10037944 0.21573223 0.0260814 ]
0.0 [[26881.23125601]] [0.10037941 0.21573241 0.02608142]
0.0 [[26881.23125587]] [0.10037942 0.21573241 0.02608142]
0.0 [[26881.23125601]] [0.10037941 0.21573242 0.02608142]
0.0 [[26881.23125605]] [0.10037941 0.21573241 0.02608143]
0.0 [[26881.23125605]] [0.10037942 0.21573237 0.02608141]
0.0 [[26881.23125589]] [0.10037943 0.21573237 0.02608141]
0.0 [[26881.23125604]] [0.10037942 0.21573238 0.02608141]
0.0 [[26881.23125604]] [0.10037942 0.21573237 0.02608142]
0.0 [[26881.2312

In [203]:
def covSEard(hyp=None, x=None, z=None):
    ''' Squared Exponential covariance function with Automatic Relevance Detemination
     (ARD) distance measure. The covariance function is parameterized as:

     k(x^p,x^q) = sf2 * exp(-(x^p - x^q)' * inv(P) * (x^p - x^q)/2)

     where the P matrix is diagonal with ARD parameters ell_1^2,...,ell_D^2, where
     D is the dimension of the input space and sf2 is the signal variance.

     The hyperparameters are:

     hyp = [ log(ell_1)
             log(ell_2)
             ...
             log(ell_D)
             log(sqrt(sf2)) ]
    '''
    #     if hyp == None:                 # report number of parameters
    #         return ['D + 1']            # USAGE: integer OR D_+_int (spaces are SIGNIFICANT)

    [n, D] = x.shape
    ell = 1/np.exp(hyp[0:D])        # characteristic length scale


    sf2 = np.exp(2.*hyp[D])         # signal variance
    tmp = np.dot(np.diag(ell),x.T).T
#         A = spdist.cdist(tmp, tmp, 'sqeuclidean')

    if z == 'itself':
        tmp = np.dot(np.diag(ell),x.T).T
        A = spdist.cdist(tmp, tmp, 'sqeuclidean')
    else:                           # compute covariance between data sets x and z
        A = spdist.cdist(np.dot(np.diag(ell),x.T).T, np.dot(np.diag(ell),z.T).T, 'sqeuclidean') # cross covariances

    A = sf2*np.exp(-0.5*A)  

    return A


#     def kernel(X1, X2, l=1.0, sigma_f=1.0):
#         '''
#         Isotropic squared exponential kernel. Computes 
#         a covariance matrix from points in X1 and X2.

#         Args:
#             X1: Array of m points (m x d).
#             X2: Array of n points (n x d).

#         Returns:
#             Covariance matrix (m x n).
#         '''
#         sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
#         return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

def posterior_predictive(X_s, X_train, Y_train, l1=0.5, l2=0.5, sigma_f=1.0, sigma_y=0):
    '''  
    Computes the suffifient statistics of the GP posterior predictive distribution 
    from m training data X_train and Y_train and n new inputs X_s.

    Args:
        X_s: New input locations (n x d).
        X_train: Training locations (m x d).
        Y_train: Training targets (m x 1).
        l: Kernel length parameter.
        sigma_f: Kernel vertical variation parameter.
        sigma_y: Noise parameter.

    Returns:
        Posterior mean vector (n x d) and covariance matrix (n x n).
    '''

#         K = kernel(X_train, X_train, l, sigma_f) + sigma_y**2 * np.eye(len(X_train))
#         K_s = kernel(X_train, X_s, l, sigma_f)
#         K_ss = kernel(X_s, X_s, l, sigma_f) + 1e-8 * np.eye(len(X_s))
#         K_inv = inv(K)


    K = covSEard(hyp=[l1,l2,sigma_f], x=X_train, z='itself') + sigma_y**2 * np.eye(len(X_train))
    K_s = covSEard(hyp=[l1,l2,sigma_f], x=X_train, z=X_s)
    K_ss = covSEard(hyp=[l1,l2,sigma_f], x=X_s, z='itself')  + 1e-8 * np.eye(len(X_s))
#         K_inv = inv(K)
    K_inv = np.linalg.pinv(K)

    # Equation (4)
    mu_s = K_s.T.dot(K_inv).dot(Y_train)

    # Equation (5)
#         print(K_s.T.dot(K_inv).shape, K_s.T.dot(K_inv).dot(K_s).shape, K_ss.shape)
    cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)

#         min_eig = np.min(np.real(np.linalg.eigvals(cov_s)))
# #         print("eig:",min_eig)
#         if min_eig < 0:
#             cov_s -= 10*min_eig * np.eye(*cov_s.shape)

#         cov_s += 1e8*min_eig * np.eye(*cov_s.shape)

    return mu_s, cov_s

In [18]:
@tf.function(autograph=False, experimental_compile=False)
def target_log_prob(amplitude, length_scale, poroi, lam):
    tf.random.set_seed(1234)
    se_kernel = tfk.ExponentiatedQuadratic(amplitude)  # length_scale = None here, implicitly
    optimized_kernel = InputScaledKernel(se_kernel, length_scale)
    gprm = tfd.GaussianProcessRegressionModel(kernel=optimized_kernel, index_points = x_unlabeled)
    samples = gprm.sample(1)
    pred = tf.squeeze(samples, axis=0)

    phyloss_poro = tf.math.reduce_mean(tf.nn.relu(tf.negative(pred))+tf.nn.relu(pred-poroi))

#     print("phyloss_poro:",lam*phyloss_poro)
#     return lam*phyloss_poro
    return lam*phyloss_poro - gp_joint_model.log_prob({
      'amplitude': amplitude,
      'length_scale': length_scale,
      'observations': trainY
  })

In [19]:
tf.random.set_seed(1234)

# Optimize the model parameters.
num_iters = 40
lam = 100000
optimizer = tf.optimizers.Adam(learning_rate=.1)

# Store the likelihood values during training, so we can plot the progress
lls_ = np.zeros(num_iters, np.float64)

for i in range(num_iters):
    with tf.GradientTape() as tape:
        loss = target_log_prob(amplitude_var, length_scale_var, init_poro, lam) # physics loss & normal loss


    print(i,"loss_inloop:",loss)
    grads = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(grads, trainable_variables))
    lls_[i] = loss

print('Trained parameters:')
print('amplitude: {}'.format(amplitude_var._value().numpy()))
print('length_scale: {}'.format(length_scale_var._value().numpy()))

0 loss_inloop: tf.Tensor(97327.17451122966, shape=(), dtype=float64)
1 loss_inloop: tf.Tensor(27517.653673109533, shape=(), dtype=float64)
2 loss_inloop: tf.Tensor(26125.867318869172, shape=(), dtype=float64)
3 loss_inloop: tf.Tensor(16993.676754054417, shape=(), dtype=float64)
4 loss_inloop: tf.Tensor(63578.97302255117, shape=(), dtype=float64)
5 loss_inloop: tf.Tensor(3150.835495812226, shape=(), dtype=float64)
6 loss_inloop: tf.Tensor(23689.326876749325, shape=(), dtype=float64)
7 loss_inloop: tf.Tensor(26075.86432005829, shape=(), dtype=float64)
8 loss_inloop: tf.Tensor(33741.71708333892, shape=(), dtype=float64)
9 loss_inloop: tf.Tensor(35355.67059762244, shape=(), dtype=float64)
10 loss_inloop: tf.Tensor(17614.334487859152, shape=(), dtype=float64)
11 loss_inloop: tf.Tensor(43646.772000166406, shape=(), dtype=float64)
12 loss_inloop: tf.Tensor(14456.26112218793, shape=(), dtype=float64)
13 loss_inloop: tf.Tensor(19869.55961840706, shape=(), dtype=float64)
14 loss_inloop: tf.Tenso

In [20]:
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp
tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels
tf.enable_v2_behavior()


import random

def pass_arg(Xx, nsim, tr_size):
    print("Tr_size:", tr_size)
    def fix_seeds(seed):
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
        sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    #     K.set_session(sess)
        tf.compat.v1.keras.backend.set_session(sess)

    ss = 1
    fix_seeds(ss)

    # Compute the RMSE given the ground truth (y_true) and the predictions(y_pred)
    def root_mean_squared_error(y_true, y_pred):
            return tf.math.sqrt(tf.math.reduce_mean(tf.math.square(y_pred - y_true), axis=-1))

    class InputTransformedKernel(tfp.math.psd_kernels.PositiveSemidefiniteKernel):

        def __init__(self, kernel, transformation, name='InputTransformedKernel'):
            self._kernel = kernel
            self._transformation = transformation
            super(InputTransformedKernel, self).__init__(
                feature_ndims=kernel.feature_ndims,
                dtype=kernel.dtype,
                name=name)

        def apply(self, x1, x2):
            return self._kernel.apply(
                self._transformation(x1),
                self._transformation(x2))

        def matrix(self, x1, x2):
            return self._kernel.matrix(
                self._transformation(x1),
                self._transformation(x2))

        @property
        def batch_shape(self):
            return self._kernel.batch_shape

        def batch_shape_tensor(self):
            return self._kernel.batch_shape_tensor

    class InputScaledKernel(InputTransformedKernel):

        def __init__(self, kernel, length_scales):
            super(InputScaledKernel, self).__init__(
                kernel,
                lambda x: x / tf.expand_dims(length_scales,
                                         -(kernel.feature_ndims + 1)))

    # Load labeled data
    data = np.loadtxt('../data/labeled_data.dat')
    x_labeled = data[:, :2].astype(np.float64) # -2 because we do not need porosity predictions
    y_labeled = data[:, -2:-1].astype(np.float64) # dimensionless bond length and porosity measurements

    # Normalize the data.
    from sklearn import preprocessing

    # normalize dataset with MinMaxScaler
    scaler = preprocessing.MinMaxScaler(feature_range=(0.0, 1.0))
    x_labeled = scaler.fit_transform(x_labeled)
    # y_labeled = scaler.fit_transform(y_labeled)

    tr_size = 30

    # train and test data
    trainX, trainY = x_labeled[:tr_size,:], y_labeled[:tr_size]
    testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]

    trainY = np.transpose(trainY)
    testY = np.transpose(testY)

    data_phyloss = np.loadtxt('../data/unlabeled_data_BK_constw_v2_1525.dat')
    x_unlabeled = data_phyloss[:, :]

    # initial porosity
    initporo = x_unlabeled[:, -1]

    x_unlabeled1 = x_unlabeled[:1303, :2]
    x_unlabeled2 = x_unlabeled[-6:, :2]
    x_unlabeled = np.vstack((x_unlabeled1,x_unlabeled2))

    x_unlabeled = scaler.fit_transform(x_unlabeled)
    init_poro1 = initporo[:1303]
    init_poro2 = initporo[-6:]
    init_poro = np.hstack((init_poro1,init_poro2))


    def build_gp(amplitude, length_scale):
        """Defines the conditional dist. of GP outputs, given kernel parameters."""

        # Create the covariance kernel, which will be shared between the prior (which we
        # use for maximum likelihood training) and the posterior (which we use for
        # posterior predictive sampling)    
        se_kernel = tfk.ExponentiatedQuadratic(amplitude)  # length_scale = None here, implicitly

        # This is the "ARD" kernel (we don't like abbreviations or bizarrely obscure names in
        # TFP, so we're probably going to call this "InputScaledKernel" since....that's what it is! :)
        kernel = InputScaledKernel(se_kernel, length_scale)

        # Create the GP prior distribution, which we will use to train the model
        # parameters.
        return tfd.GaussianProcess(kernel=kernel,index_points=trainX)

    gp_joint_model = tfd.JointDistributionNamedAutoBatched({
        'amplitude': tfd.TransformedDistribution(
                distribution=tfd.Normal(loc=0., scale=np.float64(1.)),
                bijector=tfb.Exp(),
                batch_shape=[1]),
        'length_scale': tfd.TransformedDistribution(
                distribution=tfd.Normal(loc=0., scale=np.float64(1.)),
                bijector=tfb.Exp(),
                batch_shape=[2]),
        'observations': build_gp,
    })					



    # Create the trainable model parameters, which we'll subsequently optimize.
    # Note that we constrain them to be strictly positive.
    constrain_positive = tfb.Shift(np.finfo(np.float64).tiny)(tfb.Exp())

    amplitude_var = tfp.util.TransformedVariable(
        initial_value=np.random.uniform(size=1),
        bijector=constrain_positive,
        name='amplitude',
        dtype=np.float64)

    length_scale_var = tfp.util.TransformedVariable(
        initial_value=np.random.uniform(size=[2]),
        bijector=constrain_positive,
        name='length_scale',
        dtype=np.float64)

    trainable_variables = [v.trainable_variables[0] for v in 
                           [amplitude_var,
                           length_scale_var]]



    @tf.function(autograph=False, experimental_compile=False)
    def target_log_prob(amplitude, length_scale, poroi, lam):
        tf.random.set_seed(1234)
        se_kernel = tfk.ExponentiatedQuadratic(amplitude)  # length_scale = None here, implicitly
        optimized_kernel = InputScaledKernel(se_kernel, length_scale)
        gprm = tfd.GaussianProcessRegressionModel(kernel=optimized_kernel, index_points = x_unlabeled)
        samples = gprm.sample(1)
        pred = tf.squeeze(samples, axis=0)

        phyloss_poro = tf.math.reduce_mean(tf.nn.relu(tf.negative(pred))+tf.nn.relu(pred-poroi))

    #     print("phyloss_poro:",lam*phyloss_poro)
    #     return lam*phyloss_poro
        return lam*phyloss_poro - gp_joint_model.log_prob({
          'amplitude': amplitude,
          'length_scale': length_scale,
          'observations': trainY
      })


    tf.random.set_seed(1234)

    # Optimize the model parameters.
    num_iters = 40
    lam = 100000
    optimizer = tf.optimizers.Adam(learning_rate=.1)

    # Store the likelihood values during training, so we can plot the progress
    lls_ = np.zeros(num_iters, np.float64)

    for i in range(num_iters):
        with tf.GradientTape() as tape:
            loss = target_log_prob(amplitude_var, length_scale_var, init_poro, lam) # physics loss & normal loss


        # print(i,"loss_inloop:",loss)
        grads = tape.gradient(loss, trainable_variables)
        optimizer.apply_gradients(zip(grads, trainable_variables))
        lls_[i] = loss

    # print('Trained parameters:')
    # print('amplitude: {}'.format(amplitude_var._value().numpy()))
    # print('length_scale: {}'.format(length_scale_var._value().numpy()))



    tf.random.set_seed(1234)
    se_kernel = tfk.ExponentiatedQuadratic(amplitude_var)  # length_scale = None here, implicitly
    optimized_kernel = InputScaledKernel(se_kernel, length_scale_var)
    gprm = tfd.GaussianProcessRegressionModel(kernel=optimized_kernel, index_points = testX)
    samples = gprm.sample(1)
    samples

In [28]:
tf.random.set_seed(1234)
se_kernel = tfk.ExponentiatedQuadratic(amplitude_var)  # length_scale = None here, implicitly
optimized_kernel = InputScaledKernel(se_kernel, length_scale_var)
gprm = tfd.GaussianProcessRegressionModel(kernel=optimized_kernel, index_points = testX)
samples = gprm.sample(2)
# optimized_kernel = tfk.ExponentiatedQuadratic(amplitude_var, length_scale_var)
# gpr = tfd.GaussianProcessRegressionModel(kernel=optimized_kernel, index_points = testX)
# samples = gpr.sample(1)
samples

<tf.Tensor: shape=(2, 1, 9), dtype=float64, numpy=
array([[[-0.09655712, -0.09761954, -0.06867765,  0.08486226,
          0.07861124,  0.11342095,  0.09207502, -0.07753923,
          0.03311804]],

       [[ 0.04212455,  0.03606457,  0.06086621,  0.0506872 ,
         -0.11496251, -0.09564325, -0.02192549,  0.0386103 ,
          0.02477364]]])>

In [30]:
np.array(tf.squeeze(samples, axis=1)).shape

(2, 9)

In [22]:
testY

array([[0.0105933 , 0.03315482, 0.01352198, 0.05269549, 0.00490424,
        0.00895544, 0.01003564, 0.01345683, 0.02496436]])

In [23]:
root_mean_squared_error(testY, samples)

<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[0.08679325]])>

In [24]:
import matplotlib.pyplot as plt
# Plot the loss evolution
plt.figure(figsize=(12, 4))
plt.plot(lls_)
plt.xlabel("Training iteration")
plt.ylabel("Log marginal likelihood")
plt.show()

ModuleNotFoundError: No module named 'matplotlib'